# TF Estimator

In [ ]:
from sklearn.datasets import load_wine

In [2]:
wine = load_wine()

In [3]:
type(wine)

sklearn.utils.Bunch

In [4]:
wine.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
print(wine['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [6]:
feat_data = wine['data']

In [7]:
labels = wine['target']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,labels, test_size=0.30, random_state=101)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaled_x_train = scaler.fit_transform(X_train)

In [13]:
scaled_x_test = scaler.transform(X_test)

# TF Estimator 

In [14]:
import tensorflow as tf

In [15]:
from tensorflow import estimator

In [16]:
X_train.shape

(124, 13)

In [17]:
feat_cols = [tf.feature_column.numeric_column('x', shape=[13])]

In [18]:
deep_model = estimator.DNNClassifier(hidden_units=[13,35,13],
                                     feature_columns=feat_cols,
                                     n_classes=3,
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpahxehwta', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12632f6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},
                                           y=y_train,
                                           shuffle=True,
                                           batch_size=10,
                                           num_epochs=5)

In [20]:
deep_model.train(input_fn=input_fn,steps=10000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpahxehwta/model.ckpt.
INFO:tensorflow:loss = 10.93531, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpahxehwta/model.ckpt.
INFO:tensorflow:Loss for final step: 7.710565.


In [21]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test}, shuffle=False)

In [22]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpahxehwta/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [23]:
predictions = [p['class_ids'][0] for p in preds]

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.82      0.47      0.60        19
           1       0.55      0.55      0.55        22
           2       0.62      1.00      0.76        13

    accuracy                           0.63        54
   macro avg       0.66      0.67      0.64        54
weighted avg       0.66      0.63      0.62        54

